In [1]:
!pip install yfinance

In [4]:
import yfinance as yf

In [6]:
ticker = yf.Ticker("AAPL")
hist_data = ticker.history(period="7y")

print(hist_data.head())

                                Open       High        Low      Close  \
Date                                                                    
2018-03-14 00:00:00-04:00  42.546646  42.593836  41.954408  42.103058   
2018-03-15 00:00:00-04:00  42.117204  42.527760  42.015747  42.152596   
2018-03-16 00:00:00-04:00  42.152606  42.263503  41.909577  42.003960   
2018-03-19 00:00:00-04:00  41.838804  41.874196  40.975223  41.362183   
2018-03-20 00:00:00-04:00  41.348015  41.716097  41.277229  41.348015   

                              Volume  Dividends  Stock Splits  
Date                                                           
2018-03-14 00:00:00-04:00  117473600        0.0           0.0  
2018-03-15 00:00:00-04:00   90975200        0.0           0.0  
2018-03-16 00:00:00-04:00  157618800        0.0           0.0  
2018-03-19 00:00:00-04:00  133787200        0.0           0.0  
2018-03-20 00:00:00-04:00   78597600        0.0           0.0  


In [9]:
aapl_historical = ticker.history(start="2020-06-02", end="2020-06-07", interval="1d")  # 1d for longer ticker
print(aapl_historical)

                                Open       High        Low      Close  \
Date                                                                    
2020-06-02 00:00:00-04:00  78.050005  78.704580  77.607132  78.680244   
2020-06-03 00:00:00-04:00  79.001448  79.376187  78.427171  79.113380   
2020-06-04 00:00:00-04:00  78.935745  79.235043  78.057298  78.432037   
2020-06-05 00:00:00-04:00  78.682669  80.726689  78.653470  80.665855   

                              Volume  Dividends  Stock Splits  
Date                                                           
2020-06-02 00:00:00-04:00   87642800        0.0           0.0  
2020-06-03 00:00:00-04:00  104491200        0.0           0.0  
2020-06-04 00:00:00-04:00   87560400        0.0           0.0  
2020-06-05 00:00:00-04:00  137250400        0.0           0.0  
